In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain,conversation
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain_core.prompts import PromptTemplate
import os

os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(
    model="gpt-4o-mini"
)

In [8]:
traslate_template=""""將以下中文翻譯成英文
{input}"""
write_template=""""根據以下提示創作一段文字
{input}"""

In [9]:
genral_template="""回答以下問題:
{input}"""

genral_prompt=PromptTemplate(
    template=genral_template,
    input_variables=["input"],
    output_variable=["text"]
)

genral_chain=LLMChain(llm=llm,prompt=genral_prompt)

In [10]:
prompt_infos=[
    {
        "name":"template_chain",
        "description":"將以下中文翻譯成英文",
        "prompt_template":traslate_template
    },
    {
        "name":"write_chain",
        "description":"回答以下問題",
        "prompt_template":write_template
    }

]

In [17]:
destinations=[f"{p['name']}:{p['description']}" for p in prompt_infos]

destinations_str="\n".join(destinations)

router_template=MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt=PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain=LLMRouterChain.from_llm(llm,router_prompt)


In [ ]:
destination_chains={}
for p_info in prompt_infos:
    name=p_info["name"]
    prompt_template=p_info[prompt_template]
    prompt=PromptTemplate(template=prompt_template,input_variables=["input"])
    chain=LLMChain(llm=llm,prompt=prompt)
    destination_chains[name]=chain